In [5]:
import csv
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import sklearn.utils

samples = []


with open('challenge_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:   
        samples.append(line)
        

                     
    
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
    
images = []
measurements = []


def generator(samples, batch_size):
    
    num_samples = len(samples)
    while 1:
        sklearn.utils.shuffle(samples)
        for offset in range(0,num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images =[]
            angles =[]
        
            for batch_sample in batch_samples:
    # Center Camera
                
                filename_c = 'challenge_data/IMG/'+batch_sample[0].split('\\')[-1]
                image_c = cv2.imread(filename_c)
                image_center = cv2.cvtColor(image_c, cv2.COLOR_BGR2RGB)
                image_flipped = np.fliplr(image_center)
                images.append(image_center)
                images.append(image_flipped)
    
                measurement_center = np.float64(batch_sample[3])
                measurement_center_flipped = -np.copy(measurement_center)
                angles.append(measurement_center)
                angles.append(measurement_center_flipped)
    
    # Correction
    
                correction = 0.09

    #Left Camera
    
                filename_l = 'challenge_data/IMG/'+batch_sample[1].split('\\')[-1]
                image_l = cv2.imread(filename_l)
                image_left = cv2.cvtColor(image_l, cv2.COLOR_BGR2RGB)
                images.append(image_left)
    
                measurement_left = np.float64(batch_sample[3]) + correction
                angles.append(measurement_left)
    
    #Right Camera
    
                filename_r = 'challenge_data/IMG/'+batch_sample[2].split('\\')[-1]
                image_r = cv2.imread(filename_r)
                image_right = cv2.cvtColor(image_r, cv2.COLOR_BGR2RGB)
                images.append(image_right)
    
                measurement_right = np.float64(batch_sample[3]) - correction
                angles.append(measurement_right)
    
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle (X_train, y_train)
                

In [6]:
train_generator = generator(train_samples, batch_size=10)
validation_generator = generator(validation_samples, batch_size=10)

In [7]:
print(len(train_samples))
print(len(validation_samples))

6419
1605


In [8]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Lambda, Dropout, Convolution2D, MaxPooling2D, Cropping2D

import warnings
warnings.filterwarnings('ignore')

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))

model.add(Convolution2D(24,5,5,subsample=(2,2), activation="relu")) 
model.add(Dropout(0.2))
model.add(Convolution2D(36,5,5,subsample=(2,2), activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(48,5,5,subsample=(2,2), activation="relu")) 
model.add(Dropout(0.2))

model.add(Convolution2D(64,3,3, activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(64,3,3, activation="relu"))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

#model.fit(X_train,y_train,validation_split=0.2, shuffle=True, nb_epoch=5)

model.fit_generator(train_generator, samples_per_epoch= len(train_samples),validation_data=validation_generator,nb_val_samples=len(validation_samples), nb_epoch=5)


model.save('model_challenge.h5')






Epoch 1/5
6440/6419 [==============================] - 17s - loss: 0.1463 - val_loss: 0.1219
Epoch 2/5
6440/6419 [==============================] - 17s - loss: 0.1104 - val_loss: 0.1105
Epoch 3/5
6440/6419 [==============================] - 17s - loss: 0.0996 - val_loss: 0.0895
Epoch 4/5
6436/6419 [==============================] - 17s - loss: 0.0932 - val_loss: 0.0973
Epoch 5/5
6440/6419 [==============================] - 17s - loss: 0.0929 - val_loss: 0.0907
